In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

23/03/28 13:03:41 WARN Utils: Your hostname, TIGER03531 resolves to a loopback address: 127.0.1.1; using 172.29.67.58 instead (on interface eth0)
23/03/28 13:03:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/28 13:03:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/28 13:03:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
person = spark.createDataFrame([
(0, "Bill Chambers", 0, [100]),
(1, "Matei Zaharia", 1, [500, 250, 100]),
(2, "Michael Armbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")

In [4]:
graduateProgram = spark.createDataFrame([
(0, "Masters", "School of Information", "UC Berkeley"),
(2, "Masters", "EECS", "UC Berkeley"),
(1, "Ph.D.", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")

In [5]:
sparkStatus = spark.createDataFrame([
(500, "Vice President"),
(250, "PMC Member"),
(100, "Contributor")])\
.toDF("id", "status")

In [6]:
person.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



In [43]:
graduateProgram.select(isnan('degree')).show()

+-------------+
|isnan(degree)|
+-------------+
|        false|
|        false|
|        false|
+-------------+



In [8]:
sparkStatus.show()

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+



In [9]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [10]:
joinExpression = person["graduate_program"] == graduateProgram['id']

In [11]:
wrongJoinExpression = person["name"] == graduateProgram["school"]

## Join

In [12]:
person.join(graduateProgram, joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [13]:
person.join(graduateProgram, wrongJoinExpression).show()

+---+----+----------------+------------+---+------+----------+------+
| id|name|graduate_program|spark_status| id|degree|department|school|
+---+----+----------------+------------+---+------+----------+------+
+---+----+----------------+------------+---+------+----------+------+



### Inner Join

In [14]:
person.join(graduateProgram, joinExpression, 'inner').show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



### Outer Join

In [15]:
person.join(graduateProgram, joinExpression, 'outer').show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



### Left Outer Join

In [16]:
person.join(graduateProgram, joinExpression, 'left_outer').show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



### Right Outer Join

In [17]:
person.join(graduateProgram, joinExpression, 'right_outer').show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



### Left Semi Join

In [18]:
person.join(graduateProgram, joinExpression, 'left_semi').show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



### Left Anti Join

In [19]:
person.join(graduateProgram, joinExpression, 'left_anti').show()

+---+----+----------------+------------+
| id|name|graduate_program|spark_status|
+---+----+----------------+------------+
+---+----+----------------+------------+



### Cross Join

In [20]:
person.join(graduateProgram, joinExpression, 'cross').show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



### Complex Type Join

In [21]:
from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personId")\
.join(sparkStatus, expr("array_contains(spark_status, id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



In [22]:
def merge(df1,df2,join_expr,how):
    '''
    This function merges two Snowpark Dataframes based on a join expression and type of join.
    
    Parameters
    ----------
    
    df1          : Snowpark Dataframe 1
                 : Dataframe type must be snowflake.snowpark.DataFrame

    df2          : Snowpark Dataframe 2
                 : Dataframe type must be snowflake.snowpark.DataFrame
                 
    join_expr    : The key/common columns based on which the join will occur
                 : The expression should look like > df1['col_a'] == df2['col_b'] where col_a and col_b have the same data type
                   and the user must on his/her own ensure the data belongs to the same feature.
    how          : This represent the type of join
                 : Values could be "inner", "outer", "left", "right", "cross"
    '''
    result = df1.join(df2, join_expr, how)
    return result

In [23]:
df3 = merge(person, graduateProgram, joinExpression, 'left_anti')
df3.show()

+---+----+----------------+------------+
| id|name|graduate_program|spark_status|
+---+----+----------------+------------+
+---+----+----------------+------------+



In [38]:
def freq_table(df, col_name):
    from pyspark.sql.functions import col, sum, round, count, isnan
    from pyspark.sql.window import Window
    
    freq_df = df.groupBy(col_name).count().select(col(col_name), col('count').alias('Count'))
    total = df.count()
    freq_df = freq_df.withColumn('Frequency', round(col('count'), 2))
    freq_df = freq_df.withColumn('%', round(col('count')/total*100, 2))
    freq_df = freq_df.withColumn('Cumulative Frequency', round(sum('count').over(Window.orderBy(col(col_name))).cast('float'), 2))
    freq_df = freq_df.withColumn('Cumulative %', round(sum('count').over(Window.orderBy(col(col_name))).cast('float')/total*100, 2))
    return freq_df

In [39]:
result = freq_table(graduateProgram, 'degree')
result.show()

23/03/28 14:10:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 14:10:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 14:10:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 14:10:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 14:10:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 14:10:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 1

In [40]:
from pyspark.sql.functions import count, isnan
graduateProgram.select(count(isnan('degree')).alias("Missing Values")).show()

+--------------+
|Missing Values|
+--------------+
|             3|
+--------------+



In [50]:
def missing_value_count(df):
    from pyspark.sql.functions import isnan, when, count, col
    df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()
    return